# Install all the packages

In [2]:
!pip install scipy
!pip install numpy
!pip install matplotlib
!pip install pandas

# Import all the packages

In [ ]:
import scipy
from matplotlib import pyplot as plt
import numpy as np
import timeit 
import glob
import pandas as pd
import matplotlib.pyplot as plt
import string
import time
import matplotlib.pyplot as plt
import numpy as np

# Load Data


In [ ]:
def get_test_data():
    
    files = glob.glob('./BLM_R5IM_Data/cycle/*.csv')
    selected_file = files[0]
    x_data = np.linspace(-0.5, 10.5, 2200)

    #use x_data as x-coordinate when performing calculations
    input_data = pd.read_csv(selected_file)
    data = input_data.drop(columns = input_data.columns[0]).to_numpy()
    
    return (data, x_data)



# Find the values of the integral in each bucket

In [ ]:
def find_diff_between_adj(array):
    diff_integral = np.diff(array) # take the difference bettwen each vaule 
                                   # in order to the bucket value not the vaule from the the 
    diff_intrgral = np.insert(diff_integral, 0,array[0])
    
    return diff_integral

def get_integral(value, x):
    integral = scipy.integrate.cumulative_trapezoid(value, x=x)
    
    return integral


# Interpolation function from group F

In [ ]:
def divided_diff(x, y):
    '''
    function to calculate the divided
    differences table
    '''
    n = len(y)
    coef = np.zeros([n, n])
    # the first column is y
    coef[:,0] = y
    
    for j in range(1,n):
        for i in range(n-j):
            coef[i][j] = \
           (coef[i+1][j-1] - coef[i][j-1]) / (x[i+j]-x[i])
            
    return coef

def newton_poly(coef, x_data, x):
    '''
    evaluate the newton polynomial 
    at x
    '''
    n = len(x_data) - 1 
    p = coef[n]
    for k in range(1,n+1):
        p = coef[n-k] + (x -x_data[n-k])*p
    return p

def calibration_curve_beta(data_points):
    '''
    #todo
    '''    
    x = np.array([70, 172, 374, 617, 780])
    y = np.array([2.22E-13, 2.59E-13, 4.31E-12, 1.60E-11, 3.50E-11])

    
    a_s = divided_diff(x, y)[0, :]
    x_new = np.linspace(0, 800, data_points) 
    return newton_poly(a_s, x, x_new)


data_points = 2200
x_data = np.linspace(70, 800, data_points)
x_new = np.linspace(70, 800, data_points)
calibration_data = np.abs(calibration_curve_beta(len(x_data)))

plt.plot(x_data, calibration_curve_beta(len(x_data)))
plt.grid()

In [ ]:
x = np.array([70, 172, 374, 617, 780])
y = np.array([2.22E-13, 2.59E-13, 4.31E-12, 1.60E-11, 3.50E-11])

a_s = divided_diff(x, y)[0, :]

In [ ]:
 data_points = 2366
x_data = np.linspace(-55, 800, data_points)

plt.grid()
calibration_curve = calibration_curve_beta(len(x_data))
# plt.plot(x_data, calibration_curve)
# plt.scatter(x,y)

calibration_curve = calibration_curve[366:]

start = np.full(100,calibration_curve[0])
end = np.full(100,calibration_curve[-1])
calibration_curve = np.array([*start,*calibration_curve, *end])
plt.plot(np.linspace(-.5, 10.5 , 2200), calibration_curve)

#  Function to return ISIS synchrotron beam energy at any time in the machine cycle

In [89]:
import numpy as np
import scipy 
import matplotlib.pyplot as plt
    
def synchrotron_momentum(max_E, time):
    mpeV = scipy.constants.m_p * scipy.constants.c**2 / scipy.constants.e           # Proton mass in eV
    R0 = 26                         # Mean machine radius
    n_dip = 10                      # Number of dipoles
    dip_l = 4.4                     # Dipole length

    dip_angle = 2 * np.pi / n_dip   # Dipole bending angle
    rho = dip_l / dip_angle         # Dipole radius of curvature
    omega = 2 * np.pi * 50   

    Ek = np.array([70, max_E]) * 1e6 # Injection and extraction kinetic energies 
    E = Ek + mpeV                    # Injection and extraction kinetic energies
    p = np.sqrt(E**2 - mpeV**2)      # Injection and extraction momenta

    B = p / scipy.constants.c / rho                  # Ideal magnetic field at injection and extraction energies

    Bdip = lambda t: (B[1] + B[0] - (B[1] - B[0]) * np.cos(omega * t)) / 2  # Idealised B-field variation with AC
    pdip = lambda t: Bdip(t) * rho * scipy.constants.c                                      # Momentum from B-field in MeV

    return pdip(time*1E-3)

def synchrotron_kinetic_energy(max_E, time):
    mpeV = scipy.constants.m_p * scipy.constants.c**2 / scipy.constants.e           # Proton mass in eV    
    # Relativistic Kinetic Energy = Relativistic Energy - mass
    # return (np.sqrt(synchrotron_momentum(max_E, time)**2 + mpeV**2) - mpeV) # Return array in eV
    return (np.sqrt(synchrotron_momentum(max_E, time)**2 + mpeV**2) - mpeV)/1E6 # Return array in MeVy at any time in the machine cycle

# Function to Offset data from the uknown group(need to find out)

In [ ]:
def offset_y_data(data, offset = None, n = None, SpecialPoint = None):
    '''Offsets data in y axis by designated value.
    '''

    out = data

    if offset != None:
        out = data + offset
    if SpecialPoint != None:
        out = data - SpecialPoint
        return out

    elif isinstance(n, int):
        if n > 0:
            offset = np.mean(data[:n])
        elif n < 0:
            offset = np.mean(data[n:])
        else:
            raise ValueError("Invalid value for n")
        out = data - offset
    
    return out


# Testing whether the final data is corect

In [ ]:
def convert_to_protons(blm_data, x, calibration_curve):
    #integrated_sum is an array of all integrated bins between start and stop

    k = []
    for i in range(1, 38):
        k.append(offset_y_data(blm_data[i], SpecialPoint=np.min(blm_data[i])))
    
    sum1 = np.sum(k, axis=0)

    # find integral
    integration_data = find_diff_between_adj(get_integral(x, sum1))
    final_data = integration_data / calibration_curve[:2198]
    
    return final_data 
    
def convert_to_coulombs(blm_data, x, calibration_curve):
    return convert_to_protons(blm_data, x, calibration_curve)*scipy.constants.e

def convert_to_joules(blm_data, x, calibration_curve, blm_index):
    return convert_to_protons(blm_data, x, calibration_curve)*blm_data[blm_index][:2198]

y,x = get_test_data()
plt.plot(convert_to_joules(y, x, calibration_curve, 5))